In [1]:
cd ..

/HDD/ECG_physionet


In [2]:
#### insert the name of the last run
!tensorboard --logdir=runs/May11_01-21-42_dmitrii

TensorBoard 1.15.0 at http://dmitrii:6006/ (Press CTRL+C to quit)
^C


In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import gc
import ast
import json
from data_generator import PreProcessing
from metrics import compute_beta_score

# Train

In [11]:
DATA_LIST = [
    '0_fold_0.5320328777731149.txt',
    '1_fold_0.5242880885956407.txt',
    '2_fold_0.5077841499012357.txt',
    '3_fold_0.5418115277967558.txt',
    '4_fold_0.5370379360251707.txt'
]

DEBUG_PATH = './data/CV_debug/'
DATA_PATH = './data/formatted/'

preprocessing = PreProcessing()

In [4]:
def load_data():
    
    for index,i in enumerate(DATA_LIST):
        if index == 0:
            with open(DEBUG_PATH+i, 'r') as f:
                s = f.read()
                data = ast.literal_eval(s)
        else:
            with open(DEBUG_PATH+i, 'r') as f:
                s = f.read()
                temp = ast.literal_eval(s)
            data.update(temp) 
        
    return data

data = load_data()

In [5]:
def find_optim_threshold(y_true,y_pred):
    #define a grid of thresholds
    thresholf_grid = np.arange(0.05,1.05,0.05).tolist()
    
    #list of optim thresholds for each class
    optim_threshold = []
    
    y_pred_final = y_pred.copy()
    y_pred_final[:,:] = 0
    
    
    for i in range(9): #iterate over all classes
        score_grid = []
        for threshold in thresholf_grid: #iterare over all thresholds
            temp = y_pred[:,i].copy()
            
            #apply threshold
            temp[np.where(temp >= threshold)] = 1
            temp[np.where(temp < threshold)] = 0
            
            y_pred_final[:,i] = temp
            
            f2_train, g2_train = compute_beta_score(
                    y_true, y_pred_final, beta=2, num_classes=9
                )

            score_grid.append(np.sqrt(f2_train*g2_train))
        
        optim_threshold.append(thresholf_grid[np.where(score_grid == np.max(score_grid))[0][0]])
        
        #apply optimial threshold
        temp = y_pred[:,i].copy()
        temp[np.where(temp >= optim_threshold[-1])] = 1
        temp[np.where(temp < optim_threshold[-1])] = 0
        y_pred_final[:,i] = temp
        
        
    #score_grid = np.array(score_grid)
    
    f2_train, g2_train = compute_beta_score(
                    y_true, y_pred_final, beta=2, num_classes=9
                )

    final_score = np.sqrt(f2_train*g2_train)
    
    return optim_threshold,final_score #thresholf_grid,score_grid #thresholf_grid[np.where(f1_grid == np.max(f1_grid))[0][0]]

In [6]:
predictions_raw = np.zeros((len(data.keys()),9))
y_true = np.zeros((len(data.keys()),9))

for index,i in enumerate(data.keys()):
    predictions_raw[index,:] = data[i]['preds_raw']
    y_true[index,:] = data[i]['labels']

In [7]:
find_optim_threshold(y_true,predictions_raw)

([0.35000000000000003,
  0.3,
  0.3,
  0.35000000000000003,
  0.05,
  0.05,
  0.25,
  0.15000000000000002,
  0.05],
 0.5822892196418115)

In [28]:
#calculate the score for each record
scores = np.array([])
records = np.array([]).astype(np.str)

for record in data.keys():
    scores = np.append(scores,np.sum(np.abs(np.array(data[record]['labels'],dtype=np.float) - np.array(data[record]['preds_raw'],dtype=np.float))))
    records = np.append(records,record)  
                       
#get 50 top worse examples
score_order = np.argsort(scores)
#TODO
records = records[score_order[::-1]]

In [29]:
%matplotlib qt

In [30]:
def plot_record(record,data):
    
    ecg = np.load(DATA_PATH+record+'.npy')
    ecg = preprocessing.run(ecg)
    meta = json.load(open(DATA_PATH+ f'{record}.json'))
    
    heatmap = np.array(data[record]['heatmap'],dtype=np.float)
    
    #plot the data
    fig,ax = plt.subplots(figsize=(20,20))
    fig.suptitle(record+', labels: '+str(meta['labels']) + '| labels_digit:'+str(data[record]['labels'])+'| preds_digit:'+str(data[record]['preds']))
    for i in range(12):
        ax.plot(ecg[:,i]-2.5*i)
    ax.plot(heatmap-2.5*12)
    plt.show()
    
    return 0

plot_record(record=records[1],data=data)

0

In [13]:
records[0]

'A3820'